# Descriptive Vancouver Weather Classification from Measurements

### Introduction
(Felix)

### Preliminary exploratory data analysis
(Stuart and Felix)

### Methods
(Kelly)

### Expected Outcomes and Significance
(Ina)